In [ ]:
"""
Zelfde model als op Colab (appelCNNOwnDesign.ipynb, maar iets verder uitgewerkt)

Maar dan in Torch
"""

In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils
from torch.utils.data import Dataset
import PIL
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#data voorbewerken (pixels tussen 0 en 1 plaatsen door de waardes door 255 te delen, de ToTensor klasse(?) doet dit
batch_size = 55
# Batch size in paper was 55, dus mocht ik ook 1000 fotos per klasse hebben dan staat deze al goed (anders terug op 32 zetten)
img_height = 350
img_width = 350

# voor meer transformaties in 1 keer kan je transforms.Compose([transforms.blah,transforms.blah2]) gebruiken
transform = transforms.ToTensor()
#transforms.RandomPosterize



class AppelsTrainData(Dataset):
    # hier gaan we de dataset definieren
    def __init__(self, setname):
        self.setname = setname
        # setname bepaalt of we de trainings (train) validatie (val), of test set hebben.
        # assert (nog nooit hiervoor gebruikt) checkt of de setname 1 van de 3 in de lijst is, zo ja gaat ie door anders : assertion error
        assert setname in ['train','val','test'],'Please enter one of the follow as setname: train, val, test'
        
        overall_train_dataset_dir = 'projects/apple_disease_classification/data'
        self.selected_dataset_dir = overall_train_dataset_dir,setname
        
                #E.g. self.all_filenames = ['006.png','007.png','008.png'] when setname=='val'
        self.all_filenames = os.listdir(self.selected_dataset_dir)
        self.all_labels = pd.read_csv(os.path.join(overall_dataset_dir,'tiny_labels.csv'),header=0,index_col=0)
        self.label_meanings = self.all_labels.columns.values.tolist()
        
        
    def __len__(self):
        """Return the total number of examples in this split, e.g. if
        self.setname=='train' then return the total number of examples
        in the training set"""
        return len(self.all_filenames)
    
    
    def __getitem__(self,idx):
        """Return the example at index [idx]. The example is a dict with keys
        'data' (value: Tensor for an RGB image) and 'label' (value: multi-hot
        vector as Torch tensor of gr truth class labels)."""
        selected_filename = self.all_filenames[idx]
        imagepil = PIL.Image.open(os.path.join(self.selected_dataset_dir,selected_filename)).convert('RGB')
        
        #convert image to Tensor and normalize
        image = torch.utils.to_tensor_and_normalize(imagepil)
        
        #load label
        label = torch.Tensor(self.all_labels.loc[selected_filename,:].values)
        
        sample = {'data':image, #preprocessed image, for input into NN
                  'label':label,
                  'img_idx':idx}
        return 
        
        
        
    
    




trainset = torch.utils.data.Dataset('../projects/apple_disease_classification/data', train=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torch.utils.data.Dataset(root='../projects/apple_disease_classification/data', train=False,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
# alternatief voor de loader:
# train_data_loader = torch.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True, num_workers=16) 
classes = ()

TypeError: Dataset() takes no arguments

In [8]:
    """
    Alternatieve versie, ff beide proberen en kijken wat lekkerder werkt, goede OOP oefening lijkt me zwz
    Deze komt van pytorch.org (tutorials)
    """

import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    
    # The __init__ function is run once when instantiating the Dataset object. 
    # We initialize the directory containing the images, the annotations file, and both transforms (covered in more detail in the next section).
        def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    
    # gewoon het aantal items in je set
    def __len__(self):
        return len(self.img_labels)

    # The __getitem__ function loads and returns a sample from the dataset at the given index idx. 
    # Based on the index, it identifies the image’s location on disk, converts that to a tensor using read_image, 
    # retrieves the corresponding label from the csv data in self.img_labels, calls the transform functions on them (if applicable), 
    # and returns the tensor image and corresponding label in a tuple.
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    


In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

NameError: name 'training_data' is not defined

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
class AppelClassifier(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()

        self.c1 = nn.Conv2d(in_channels=1,out_channels=128,kernel_size=3,stride=1,padding=0)
        self.c2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.c3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,stride=1,padding=0)
        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120,out_features=84)
        self.fc2 = nn.Linear(in_features=84,out_features=10)

    def forward(self, img):
        x = self.c1(img)
        x = self.relu(self.max_pool(x))
        x = self.c2(x)
        x = self.relu(self.max_pool(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x,1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x